# Alternative names (combinations) for top Colombian authors

In [1]:
import ast
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [2]:
%%time

autores_df = pd.read_csv(
    'autores_colombianos_1.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 237 ms, sys: 3.89 ms, total: 240 ms
Wall time: 239 ms


In [3]:
autores_df.shape

(5551, 12)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO
0,En la laguna más profunda,Colombia,Oscar Collazos,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bahía Solano,Chocó Department,True
1,El gato bandido y otros cuentos,Colombia,Rafael Pombo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
2,Cuentos pintados,Colombia,Rafael Pombo,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
3,Aleida : 10 años,Colombia,Vladdo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
4,Mitos y leyendas indígenas de Colombia,Colombia,Javier Ocampo López,Colombia,2013,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False


## Autores no encontrados

In [5]:
autores_no_encontrados_df = autores_df[autores_df['ENCONTRADO'] == False]

In [6]:
autores_no_encontrados_df.shape

(2315, 12)

In [7]:
autores_no_encontrados_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO
1,El gato bandido y otros cuentos,Colombia,Rafael Pombo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
2,Cuentos pintados,Colombia,Rafael Pombo,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
3,Aleida : 10 años,Colombia,Vladdo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
4,Mitos y leyendas indígenas de Colombia,Colombia,Javier Ocampo López,Colombia,2013,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
5,Secretos de el dorado : Colombia,Colombia,Germán Arciniegas,Colombia,1990,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False


In [8]:
count_autores_no_encontrados_sr = \
    autores_no_encontrados_df.groupby(['AUTOR'])['TITULO'].count().sort_values(ascending=False)

In [9]:
count_autores_no_encontrados_sr.head()

AUTOR
José María Vargas Vila        103
Germán Arciniegas             102
Eduardo Caballero Calderón     94
Javier Ocampo López            85
Rafael Pombo                   84
Name: TITULO, dtype: int64

In [10]:
THRESHOLD = 2

In [11]:
top_autores_no_encontrados_sr = count_autores_no_encontrados_sr[lambda x: x >= THRESHOLD]

In [12]:
top_autores_no_encontrados_sr.shape

(174,)

In [13]:
top_autores_no_encontrados_sr

AUTOR
José María Vargas Vila               103
Germán Arciniegas                    102
Eduardo Caballero Calderón            94
Javier Ocampo López                   85
Rafael Pombo                          84
Juan Gustavo Cobo Borda               79
Celso Román                           68
Jorge Isaacs                          65
Andrés Caicedo Estela                 51
Alfonso López Michelsen               47
Alfredo Iriarte                       40
Daniel Samper Pizano                  40
Eduardo Lemaitre Román                39
Jorge Eliécer Gaitán                  39
Jorge Zalamea                         35
Germán Colmenares                     34
Indalecio Liévano Aguirre             32
Samael Aun Weor                       32
Carlos Lleras Restrepo                30
Laura Restrepo                        29
Guillermo Abadía Morales              28
Laureano Gómez                        25
Santiago Gamboa                       25
Jorge Alí Triana                      24
Alberto Ll

## País autor

In [14]:
def get_ciudad_departamento_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?pobLabel ?depLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            OPTIONAL {{
                ?author wdt:P106 wd:Q36180 .
            }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P31 wd:Q2555896 .
            ?pob rdfs:label ?pobLabel .
            FILTER(lang(?pobLabel)="{1}")
            OPTIONAL {{
                ?pob wdt:P131 ?dep .
                ?dep wdt:P31 wd:Q215655 .
                ?dep rdfs:label ?depLabel .
                FILTER(lang(?depLabel)="{1}")
            }}
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [15]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_ciudad_departamento_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX yago: <http://dbpedia.org/class/yago/>

        SELECT ?authorLabel ?pobLabel ?depLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            OPTIONAL {{
                ?author rdf:type dbo:Writer .
            }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type yago:WikicatMunicipalitiesOfColombia .
            ?pob rdfs:label ?pobLabel .
            FILTER(lang(?pobLabel)="{1}")
            OPTIONAL {{
                ?pob dbo:isPartOf ?dep .
                ?dep rdf:type yago:WikicatDepartmentsOfColombia .
                ?dep rdfs:label ?depLabel .
                FILTER(lang(?depLabel)="{1}")
            }}
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [16]:
error_autores = set()

def get_ciudad_departamento_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_ciudad_departamento_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_ciudad_departamento_autor_data_dbp(name, lang)

        pobs = set(d['pobLabel']['value'] for d in data['results']['bindings'])
        if len(pobs) == 0:
            pob = 'UNKNOWN'
        elif len(pobs) == 1:
            pob = pobs.pop()
        else:
            print('MáS DE UNA POBLACIóN PARA', name, ':', pobs)
            pob = pobs.pop()

        deps = set(d['depLabel']['value'] for d in data['results']['bindings'])
        if len(deps) == 0:
            dep = 'UNKNOWN'
        elif len(deps) == 1:
            dep = deps.pop()
        else:
            print('MáS DE UN DEPARTAMENTO PARA', name, ':', deps)
            dep = deps.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        pob = 'UNKNOWN'
        dep = 'UNKNOWN'
        error_autores.add(name)
    return (pob, dep)

In [17]:
autores_encontrados_list = []

def get_ciudad_departamento_autor(name):
    result = ('DESCONOCIDO', 'DESCONOCIDO')
    for lang in ['es', 'en']:
        for db in ['wdt', 'dbp']:
            r = get_ciudad_departamento_autor_data(db, name, lang)
            if r[0] != 'UNKNOWN':
                autores_encontrados_list.append(name)
                if r[1] == 'UNKNOWN':
                    result = (r[0], 'DESCONOCIDO')
                else: #r[1] != 'UNKNOWN'
                    result = r
                    return result
    return result

In [18]:
%%time

autores_ciudades_dict = {}
autores_departamentos_dict = {}

for i, x in autores_df.drop_duplicates(['AUTOR']).iterrows():
    autores_ciudades_dict[x['AUTOR']] = x['CIUDAD_AUTOR']
    autores_departamentos_dict[x['AUTOR']] = x['DEPARTAMENTO_AUTOR']

CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 102 ms


In [19]:
def combinaciones(ls):
    r = []
    for x in ls:
        aux = list(ls)
        aux.remove(x)
        r.append(aux)
    return r

In [20]:
%%time

nombres_encontrados_dict = {}

for autor in top_autores_no_encontrados_sr.index:
    nombres = autor.split()
    if len(nombres) > 1:
        for c in combinaciones(nombres):
            aux = ' '.join(c)
            (ciudad, departamento) = get_ciudad_departamento_autor(aux)
            if ciudad != 'DESCONOCIDO':
                autores_ciudades_dict[autor] = ciudad
                autores_departamentos_dict[autor] = departamento
                nombres_encontrados_dict[autor] = aux
                break

CPU times: user 27.7 s, sys: 1.05 s, total: 28.8 s
Wall time: 7min 41s


In [21]:
%%time

def process_ciudad(x):
    return autores_ciudades_dict[x['AUTOR']]

autores_df['CIUDAD_AUTOR'] = autores_df.apply(process_ciudad, axis=1)

CPU times: user 155 ms, sys: 0 ns, total: 155 ms
Wall time: 153 ms


In [23]:
%%time

def process_departamento(x):
    return autores_departamentos_dict[x['AUTOR']]

autores_df['DEPARTAMENTO_AUTOR'] = autores_df.apply(process_departamento, axis=1)

CPU times: user 160 ms, sys: 0 ns, total: 160 ms
Wall time: 159 ms


In [25]:
%%time

def process_nombre_encontrado(x):
    if x['ENCONTRADO']:
        r = x['AUTOR']
    elif x['AUTOR'] in nombres_encontrados_dict:
        r = nombres_encontrados_dict[x['AUTOR']]
    else:
        r = ''
    return r

autores_df['NOMBRE_ENCONTRADO'] = autores_df.apply(process_nombre_encontrado, axis=1)

CPU times: user 260 ms, sys: 0 ns, total: 260 ms
Wall time: 259 ms


In [28]:
%%time

def process_encontrado(x):
    return x['ENCONTRADO'] or x['AUTOR'] in nombres_encontrados_dict

autores_df['ENCONTRADO'] = autores_df.apply(process_encontrado, axis=1)

CPU times: user 206 ms, sys: 41 µs, total: 206 ms
Wall time: 205 ms


# Saving results

In [29]:
autores_df.to_csv('autores_colombianos_2.csv', index=False)